# Commands

# Init

In [1]:
isNotebook = '__file__' not in locals()

In [23]:
TEST = isNotebook

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *

In [3]:
logger = Logger(tmpDir('logs') + "/lda.log") if isNotebook else Logger("lda.log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [4]:
newsCollection = getNewsCollection(logger=logger)
usersCollection = getUsersCollection(logger=logger)

Unable to create index url in twinews news
twinews news (version 1.0) initialised.
Unable to create index user_id in twinews users
twinews users (version 1.0) initialised.


In [141]:
evalData = getEvalData(1)

In [8]:
bp(evalData.keys(), 5, logger)
log(b(evalData['stats']), logger)

{ candidates, created, ranksLength, stats, testNews, testUsers, trainNews, trainUsers }
{ testMaxNewsPerUser: 164, testMeanNewsPerUser: 10.67, testMinNewsPerUser: 2, testNewsCount: 138785, totalNewsAvailable: 570210, trainMaxNewsPerUser: 443, trainMeanNewsPerUser: 28.0, trainMinNewsPerUser: 8, trainNewsCount: 356150, usersCount: 21239 }


# Sub-sampling of users and news

In [131]:
def subsampleTwinews(evalData, maxUsers=100):
    # Getting a sub-sample of user ids:
    userIds = random.sample(list(evalData['testUsers'].keys()), usersCount)
    # Sub-sampling users:
    trainUsers = dictSelect(evalData['trainUsers'], userIds)
    testUsers = dictSelect(evalData['testUsers'], userIds)
    # Sub-sampling candidates:
    candidates = dictSelect(evalData['candidates'], userIds)
    for userId in candidates:
        cNews = set(candidates[userId][0])
        print(len(cNews))
        trNews = set(trainUsers[userId].keys())
        print(len(trNews))
        teNews = set(testUsers[userId].keys())
        for n in teNews:
            if n not in cNews:
                print(n)
        print(len(teNews))
        
        print()
    # Getting urls:
    urls = set()
    for users in (trainUsers, testUsers):
        for userId, news in users.items():
            for n in news.keys():
                urls.add(n)
    for userId, bulks in candidates.items():
        for news in bulks:
            for n in news:
                urls.add(n)
    print(len(urls))
    # Sub-sampling news:
    trainNews = set([n for n in evalData['trainNews'] if n in urls])
    testNews = set([n for n in evalData['testNews'] if n in urls])
    # We return all sub samples:
    return (trainUsers, testUsers, trainNews, testNews, candidates)

In [132]:
if TEST:
    usersCount = 2
    (trainUsers, testUsers, trainNews, testNews, candidates) = subsampleTwinews(evalData, maxUsers=100)
else:
    (trainUsers, testUsers, trainNews, testNews, candidates) = \
    (evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
     evalData['testNews'], evalData['candidates'])

1000
8
4

1000
32
11

2032


In [145]:
nbNews = 0
for users in (trainUsers, testUsers):
    for userId, news in users.items():
        nbNews += len(news)
print(nbNews)

821524


In [147]:
newsSet = set()
for users in (trainUsers, testUsers):
    for userId, news in users.items():
        for n in news:
            newsSet.add(n)
print(len(newsSet))

450310


In [148]:
len(trainNews) + len(testNews)

494935

In [149]:
len(trainNews.union(testNews))

494935

In [137]:
newsSet = set()
for userId, news in candidates.items():
    news = news[0]
    for n in news:
        newsSet.add(n)
print(len(newsSet))

1992


In [138]:
newsCount = 0
for userId, news in candidates.items():
    news = news[0]
    for n in news:
        newsCount += 1
print(newsCount)

2000


In [139]:
newsSet = set()
for users in (trainUsers,):
    for userId, news in users.items():
        for n in news:
            newsSet.add(n)
print(len(newsSet))

40


In [140]:
newsCount + len(newsSet)

2040

In [142]:
(trainUsers, testUsers, trainNews, testNews, candidates) = \
    (evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
     evalData['testNews'], evalData['candidates'])

In [143]:
candidatesUrls = set()
for userId, news in candidates.items():
    for n in news[0]:
        candidatesUrls.add(n)
print(len(candidatesUrls))

138785


In [144]:
print(len(testNews))

138785


In [41]:
len(trainUsers[trainUsers.keys()]) + len(trainUsers['2966358699']) + len(testUsers['2966358699']) + len(testUsers['2966358699'])

KeyError: '162158138'

2025

In [34]:
bp(trainUsers)

{
  162158138: 
  {
    http://fb.me/2Ka6jIF6p: 1510704050,
    http://fb.me/2pLvGDQPy: 1512325824,
    http://ow.ly/4MYb30gHGLZ: 1511373654,
    http://ow.ly/AfoW30gJvdN: 1511305225,
    http://ow.ly/BSrf30hdj14: 1513215129,
    ...,
    http://ow.ly/vklJ30gA8XA: 1510698003,
    http://ow.ly/wNPE30gSBcH: 1511910696,
    http://ow.ly/yigL30h5tai: 1512688194,
    http://wapo.st/2zKew8G?tid=ss_tw-bottom&utm_term=.cbd1e89762fa: 1511217021,
    http://www.news.ucsb.edu/2017/018475/resisting-alien-invasions: 1509831017
  },
  2966358699: 
  {
    http://fb.me/1s4upTuaY: 1512141917,
    http://ow.ly/3uR330gkuaB: 1509713413,
    http://ow.ly/T3bS30gcDov: 1509278950,
    http://wane.com/2017/11/11/scott-sycamore-spoil-archie-millers-iu-debut/: 1510378613,
    http://wane.com/2017/12/18/12-18-wendys-play-of-the-week/: 1513660995,
    http://www.news-sentinel.com/news/local-news/2017/11/07/fort-wayne-community-schools-showcase-offers-chance-to-learn-more-about-school-options/: 1510066767,
    ht

In [26]:
bp(testUsers)

{
  24292383: 
  {
    http://www.montrosepress.com/breaking_news/multiple-willful-violations/article_74375056-113f-11e8-9ad0-1bfdf00eb817.html#utm_campaign%3Dblox%26utm_source%3Dtwitter%26utm_medium%3Dsocial: 1518582686,
    http://www.montrosepress.com/breaking_news/police-standoff-in-progress/article_633623d8-03c3-11e8-9d85-13ee877ab253.html#utm_campaign%3Dblox%26utm_source%3Dtwitter%26utm_medium%3Dsocial: 1517100372,
    http://www.montrosepress.com/free_access/black-canyon-open-during-shutdown/article_ac571210-ff9b-11e7-b96d-ab35f0790d00.html#utm_campaign%3Dblox%26utm_source%3Dtwitter%26utm_medium%3Dsocial: 1516643372,
    http://www.montrosepress.com/free_access/coroner-traumatic-injury-hemorrhage-claimed-olathe-man/article_964e6a54-1274-11e8-ab42-ab610d4c2fb6.html#utm_campaign%3Dblox%26utm_source%3Dtwitter%26utm_medium%3Dsocial: 1518715604,
    http://www.montrosepress.com/free_access/funeral-home-director-disputes-report/article_e15e5a46-fbee-11e7-9331-7f11f1c1c3af.html#utm_cam

In [27]:
bp(trainNews)

{
  http://bit.ly/2BPeUBR,
  http://cnn.it/2EAh08W,
  ...,
  https://www.washingtonpost.com/news/wonk/wp/2017/12/01/gop-eyes-post-tax-cut-changes-to-welfare-medi,
  https://www.washingtonpost.com/politics/trump-attacks-protections-for-immigrants-from-shithole-count
}


In [29]:
bp(candidates)

{
  24292383: 
  [
    {
      http://a.msn.com/00/en-us/BBITBYi?ocid=st,
      http://a.msn.com/01/en-ca/BBJ2t9t?ocid=st,
      ...,
      https://www.yahoo.com/amphtml/sports/rockets-clippers-turned-monday-night-raw-fight-nearly-spilled-l,
      https://www.zerohedge.com/news/2018-02-13/commodity-traders-blame-data-overload-rash-fund-closures
    }
  ],
  44493998: 
  [
    {
      http://a.msn.com/0C/en-us/BBIEr1f?ocid=st,
      http://abcn.ws/2CdWoSk,
      ...,
      https://www.wthr.com/article/freezing-rain-sleet-and-snow-create-slick-road-conditions,
      https://www.zerohedge.com/news/2018-02-08/inflation-alert-velocity-money-has-finally-bottomed
    }
  ]
}
